In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

# dataset

In [ ]:
data_campur = {
    'Height': [175, 160, np.nan, 180, 165, 170, np.nan, 155, 180, 175],  # Continuous
    'Weight': [70, np.nan, 65, 80, np.nan, 75, 85, 90, np.nan, 68],      # Continuous
    'ShoeSize': [42, 38, 40, np.nan, 41, 39, 44, 37, 42, 43],            # Continuous
    'Gender': ['Male', 'Female', 'Female', 'Male', np.nan, 'Female', 'Male', 'Female', 'Male', 'Male'],  # Categorical
    'City': ['New York', 'Los Angeles', np.nan, 'Chicago', 'Houston', 'Phoenix', 'Philadelphia', np.nan, 'San Antonio', 'San Diego'],  # Categorical
    'Age': [25, 30, 22, 40, 35, np.nan, 29, 31, 28, 33],                 # Continuous
    'MaritalStatus': ['Single', 'Married', 'Single', np.nan, 'Married', 'Single', 'Single', 'Married', np.nan, 'Single'],  # Categorical
    'Salary': [50000, 55000, np.nan, 60000, 58000, 62000, 65000, np.nan, 57000, 59000]  # Continuous
}

# Membuat DataFrame dengan kolom campuran (continuous dan categorical)
gabung1 = pd.DataFrame(data_campur)

# Menampilkan DataFrame dengan nilai NaN
print(gabung1)

In [ ]:
columns_with_number = [col for col in gabung1 if gabung1[col].dtype in ['int64', 'float64']]
columns_with_word = [col for col in gabung1 if gabung1[col].dtype in ['bool', 'object']]
gabung1[columns_with_word]

# fungsi knn data gabung1 yg nilainya nan namun hanya menghitung jarak antar kolom categorikal saja

In [ ]:
from tqdm import tqdm
import numpy as np
import pandas as pd

def hamming_distance(gabung1, columns_with_word):
    # Cari baris yang memiliki nilai NaN di gabung1
    original_row_indices = gabung1.index[gabung1.isnull().any(axis=1)].tolist()
    
    # Buat matrix jarak Hamming hanya untuk baris yang memiliki nilai NaN di gabung1 dan kolom columns_with_word
    distance_matrix = np.zeros((len(original_row_indices), len(gabung1)))
    
    # Vectorized calculation of Hamming distance
    for i, nan_row in enumerate(tqdm(original_row_indices, desc="Calculating Hamming distance")):
        distance_matrix[i] = (gabung1[columns_with_word].ne(gabung1.loc[nan_row, columns_with_word], axis=1).sum(axis=1))
    
    # Set diagonal elements to 0
    distance_matrix[np.arange(len(original_row_indices)), original_row_indices] = 0
    
    # Convert distance matrix to dataframe
    df_result = pd.DataFrame(distance_matrix, index=original_row_indices, columns=gabung1.index)
    
    return df_result

distance_categorikal = hamming_distance(gabung1, columns_with_word)
print(distance_categorikal)

# fungsi knn data gabung1 yg nilainya nan namun hanya menghitung jarak antar kolom continousnya saja

In [ ]:
from sklearn.preprocessing import MinMaxScaler
import numpy as np
import pandas as pd
from tqdm import tqdm

scaler = MinMaxScaler()
X_scaled = scaler.fit_transform(gabung1[columns_with_number])

# Fungsi untuk menghitung Euclidean Distance dengan NaN dianggap sebagai 1
def euclidean_distance(x1, x2):
    diff = np.abs(x1 - x2)    
    diff[np.isnan(diff)] = 1    
    diff[np.isnan(x1) & np.isnan(x2)] = 1  # Jika keduanya NaN, jaraknya dianggap 0    
    distance = np.nansum(diff ** 2)    
    return distance

def calculate_distances(X, nan_rows, original_df):
    num_rows = X.shape[0]
    distances = np.zeros((len(nan_rows), num_rows))
    
    for i, nan_row in enumerate(tqdm(nan_rows, desc="Calculating distances for NaN rows")):
        for j in range(num_rows):
            distances[i, j] = euclidean_distance(X[nan_row], X[j])
    
    return distances

# Cari baris yang memiliki nilai NaN di gabung1
nan_rows = gabung1.index[gabung1.isnull().any(axis=1)].tolist()

# Menghitung jarak antar baris setelah di-scaler hanya untuk baris dengan NaN di gabung1 dan kolom columns_with_number
distances = calculate_distances(X_scaled, nan_rows, gabung1)

# Konversi hasil jarak ke DataFrame
distances_continous = pd.DataFrame(distances, index=nan_rows, columns=gabung1.index)

# Set value to 0 when row index matches column index
for i in distances_continous.index:
    distances_continous.loc[i, i] = 0

# Tampilkan hasil jarak antar baris dalam DataFrame
print("\nJarak Euclidean antar baris continous setelah di-scaler (dengan aturan NaN):")
print(distances_continous)


# gabungin dan mengisi row yg hilang dengan 0

In [ ]:
distance_matrix=distances_continous+distance_categorikal
print(distance_matrix)
distance_matrix =distance_matrix.map(np.sqrt)
print('\n hasil pengakaran')
print(distance_matrix)
distance_matrix = distance_matrix.reindex(gabung1.index, fill_value=0)
print('isi nilai dengan 0')
print(distance_matrix)
distance_matrix=distance_matrix.to_numpy()

# input knn

In [ ]:
df =gabung1

def knn_impute(df, distance_matrix, k=2):
    imputed_data = df.copy()
    col_name = df.columns[0]
    for i in range(len(df)):
        if np.isnan(df.iloc[i][col_name]):
            print(f"Imputing missing value at row {i}, column {col_name}...")
            distances = distance_matrix[i]
            neighbors = np.argsort(distances)[1:]  # exclude self
            print(f"Neighbors: {neighbors}")
            print(f"Neighbor values:")
            neighbor_values = []
            for neighbor in neighbors:
                value = df.iloc[neighbor, 0]
                print(f"  Neighbor {neighbor}: {value}")
                if not np.isnan(value):
                    neighbor_values.append((neighbor, value))
                if len(neighbor_values) == k:
                    break
            print(f"Using neighbors: {[neighbor for neighbor, _ in neighbor_values]}")
            if neighbor_values:
                imputed_value = np.mean([value for _, value in neighbor_values])
                print(f"Imputed value: {imputed_value}")
                imputed_data.iloc[i, 0] = imputed_value
            else:
                print("No non-missing neighbors, recursively calling the function...")
                imputed_data.iloc[neighbors, :] = knn_impute(imputed_data.iloc[neighbors, :], distance_matrix, k)
                imputed_data.iloc[i, 0] = np.nanmean(imputed_data.iloc[neighbors, 0])
                print(f"Imputed value: {imputed_data.iloc[i, 0]}")
            print()
    return imputed_data

def knn_impute_categorical(df, distance_matrix, k=3):
    imputed_data = df.copy()
    col_name = df.columns[0]
    for i in range(len(df)):
        if df.iloc[i][col_name] is np.nan:
            print(f"Imputing missing value at row {i}, column {col_name}...")
            
            neighbors = np.argsort(distance_matrix[i])[:k+1][1:]  # exclude self
            print(f"Neighbors: {neighbors}")
            # Get categorical values from neighbors
            neighbor_values = [df.iloc[neighbor, 0] for neighbor in neighbors]
            print("Neighbor values:")
            for neighbor, value in zip(neighbors, neighbor_values):
                print(f"  Neighbor {neighbor}: {value}")
            # Remove NaN values from neighbor_values
            non_nan_values = [(value, distance_matrix[i, neighbors[j]]) for j, value in enumerate(neighbor_values) if value is not np.nan]
            if non_nan_values:
                # Count the occurrences of each value
                value_counts = {}
                for value, _ in non_nan_values:
                    value_counts[value] = value_counts.get(value, 0) + 1
                # Find the mode
                max_count = max(value_counts.values())
                modes = [value for value, count in value_counts.items() if count == max_count]
                if len(modes) == 1:
                    # If there's only one mode, use it
                    imputed_value = modes[0]
                else:
                    # If there are multiple modes, use the one with the closest neighbor
                    closest_mode = min([(value, distance) for value, distance in non_nan_values if value in modes], key=lambda x: x[1])
                    imputed_value = closest_mode[0]
            else:
                # If all neighbors have NaN values, impute with the most frequent value in the column
                imputed_value = df[col_name].value_counts().index[0]
            print(f"Using neighbors: {neighbors}")
            print(f"Imputed value: {imputed_value}")
            print()
            imputed_data.iloc[i, 0] = imputed_value
    return imputed_data
imputed_df = df.copy()
for col in df.columns:
    if df[col].dtype.kind in ['i', 'f']:  # continuous column
        imputed_df[col] = knn_impute(imputed_df[[col]], distance_matrix, k=4)[col]
    else:  # categorical column
        imputed_df[col] = knn_impute_categorical(imputed_df[[col]], distance_matrix, k=4)[col]
print("Imputed dataframe:")
print(imputed_df)